In [0]:
#@title
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision torchtext
import torch

from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

tcmalloc: large alloc 1073750016 bytes == 0x5906a000 @  0x7f3b899532a4 0x591a07 0x5b5d56 0x502e9a 0x506859 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x504c28 0x502540 0x502f3d 0x507641
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
#@title
!python -m spacy download en
!pip install msgpack==0.5.6
!pip install spacy==2.0.0


    Linking successful
    /usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
    /usr/local/lib/python3.6/dist-packages/spacy/data/en

    You can now load the model via spacy.load('en')

    100% |████████████████████████████████| 13.2MB 3.3MB/s 
    100% |████████████████████████████████| 81kB 22.6MB/s 
    100% |████████████████████████████████| 1.2MB 13.0MB/s 
    100% |████████████████████████████████| 604kB 20.9MB/s 
    100% |████████████████████████████████| 51kB 19.1MB/s 
    100% |████████████████████████████████| 143kB 26.9MB/s 
  Running setup.py bdist_wheel for spacy ... - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - done
  Stored in directory: /root/.cache/pip/wheels/ae/7a/c1/48ed01646ebbacb5cce1cec6cc70870

In [0]:
!pip show spacy
!pip show msgpack

In [0]:
# DATA LOADER

import pandas as pd
import numpy as np
import os
from torchtext import data, vocab
import torch
import spacy
from random import randint

spacy_en = spacy.load('en')
from spacy.lang.en.stop_words import STOP_WORDS


device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')


def tokenizer(text):  # create a tokenizer function
  turncate = 100
  
  tokens = [tok.text for tok in spacy_en.tokenizer(text)]
  tokens = list(filter(lambda token: token not in STOP_WORDS, tokens))
  if 0 < turncate < len(tokens):
    return tokens[:turncate]
  return tokens


class citeulike:
    """
    Class to handle the Cite-U-Like data

    Predict:
    match_status

    """

    def __init__(self, batch_size=100):
        print('Device: ' + str(device))

        self.user = data.Field(sequential=False, use_vocab=False, dtype=torch.int64)
        self.doc_title = data.Field(sequential=True, lower=True, tokenize=tokenizer, include_lengths=True)
        self.ratings = data.Field(sequential=False, use_vocab=False)
        # self.doc_abstract = data.Field(sequential=True, tokenize=tokenizer, lower=True)

        self.train_set, self.validation_set, self.test_set = data.TabularDataset.splits(
            path='./gdrive/My Drive/Colab Notebooks/data/',
            train='train_data.csv',
            validation='val_data.csv',
            test='test_data.csv',
            format='csv',
            fields=[
                ('index', None),
                ('user', self.user),
                ('doc_id', None),
                ('ratings', self.ratings),
                # ('REAL_DOC_TITLE', None), # comment this line out to use the doc title
                ('doc_title', self.doc_title), 
                # ('doc_abstract', self.doc_abstract)
            ],
            skip_header=True,
        )

        self.train_iter, self.validation_iter, self.test_iter = data.BucketIterator.splits(
            (self.train_set, self.validation_set, self.test_set),
            batch_size=batch_size,
            shuffle=True,
            device=device,
            sort_key=lambda x: len(x.doc_title),
            sort_within_batch=True,
            repeat=False)

        self.user.build_vocab(self.train_set)
        self.ratings.build_vocab(self.train_set)
        url = 'https://s3-us-west-1.amazonaws.com/fasttext-vectors/wiki.simple.vec'
        # self.doc_abstract.build_vocab(self.train_set, max_size=None, vectors=vocab.Vectors('wiki.simple.vec', url=url))
        self.doc_title.build_vocab(self.train_set, max_size=None, vectors=vocab.Vectors('wiki.simple.vec', url=url))

In [0]:
# MODEL

import torch
import torch.nn as nn
import numpy as np
import torch.nn.utils.rnn as rnn


class CollaborativeFilteringNet(nn.Module):

    def __init__(self, article_field, user_field, author_dim=20, p1=0, p2=0):

        super(CollaborativeFilteringNet, self).__init__()

        article_vectors = article_field.vocab.vectors
        num_embeddings = article_vectors.size()[0]
        self.embedding_dim = article_vectors.size()[1]

        self.article_embeddings = nn.Embedding(num_embeddings, self.embedding_dim)
        self.article_embeddings.weight.data.copy_(article_vectors)
        self.article_embeddings.requires_grad = False

        num_author = len(user_field.vocab.freqs)
        self.author_embedding = nn.Embedding(num_author, author_dim)
        self.author_embedding.weight.data.uniform_(0, 0.01)
        self.author_embedding.requires_grad = False

        self.l_text = nn.Sequential(
            nn.Dropout(p1),
            nn.Linear(in_features=(self.embedding_dim),
                   out_features=author_dim,
                   bias=True),
            nn.ReLU(),
        )

        self.l_out = nn.Sequential(
            nn.Dropout(p2),
            nn.Linear(in_features=author_dim * 2,
                   out_features=1,
                   bias=True),
        )

    def forward(self, x):
        author = self.author_embedding(x.user)
        text = x.doc_title
        
        numpy_text = text.cpu().data.numpy()
        num_non_ones = np.count_nonzero(np.subtract(numpy_text, np.ones(numpy_text.shape)), axis=0)
        num_non_ones = np.repeat(np.expand_dims(num_non_ones, 1), self.embedding_dim, 1)
        num_non_ones = torch.tensor(num_non_ones).to(device).float()
        
        text = self.article_embeddings(text)
        text = torch.sum(text, 0).to(device) / num_non_ones
        text = self.l_text(text)
        
        x = torch.cat((author, text), 1)
        x = self.l_out(x)
                
        return torch.sigmoid(x)


class LstmNet(nn.Module):
    def __init__(self, article_field, user_field, user_dim=50, hidden_dim=50, lstm_layers=5):
        super(LstmNet, self).__init__()
        article_vectors = article_field.vocab.vectors
        num_embeddings = article_vectors.size()[0]
        embedding_dim = article_vectors.size()[1]

        self.article_embeddings = nn.Embedding(num_embeddings, embedding_dim)
        self.article_embeddings.weight.data.copy_(article_vectors)

        num_author = len(user_field.vocab.freqs)
        self.author_embedding = nn.Embedding(num_author, user_dim)
        self.author_embedding.weight.data.uniform_(0, 0.01)

        self.hidden_dim = hidden_dim
        self.lstm_layers = lstm_layers
        self.lstm = nn.LSTM(input_size=embedding_dim, hidden_size=hidden_dim, num_layers=lstm_layers)

        self.linear = nn.Sequential(
            nn.Linear(in_features=(user_dim + hidden_dim),
                      out_features=1,
                      bias=True),
            nn.ReLU(),
        )

    def forward(self, x):
        batch_size = len(x.user)
        user = self.author_embedding(x.user)
        text = self.article_embeddings(x.doc_title)

        ## Packing and padding
        # packed = rnn.pack_padded_sequence(text, lengths)
        lstm_out, (lstm_hidden, lstm_state) = self.lstm(text)
        # padded, lengths = rnn.pad_packed_sequence(lstm_out)


        # x = torch.cat((user, lstm_state[-1]), 1).cuda()
        # x = self.linear(x)

        x = (user * lstm_state[-1]).sum(1)

        out = torch.sigmoid(x)

        return out


In [0]:
# TRAIN

from types import SimpleNamespace
import torch
from torch.autograd import Variable
import numpy as np
import matplotlib.pyplot as plt
from random import randint


def train_with_negative_sampling(train_iter, val_iter, net, test_iter, optimizer, criterion, num_epochs=5):
    train_loss = []
    train_error = []
    train_accs = []
    val_res = []
    for i in range(num_epochs):
        net.train()
        for batch in train_iter:
            users,(docs, lengths), ratings = batch.user, batch.doc_title, batch.ratings
            net.train()

            batch_with_negative_sampling = {'user': users, 'doc_title': docs}
            output = net(SimpleNamespace(**batch_with_negative_sampling)).reshape(-1)
            targets = ratings.float().to(device)
            batch_loss = criterion(output, targets)

            train_loss.append(get_numpy(batch_loss))
            train_error.append(accuracy_sigmoid(output, targets))
            train_accs.append(accuracy(output, targets))

            optimizer.zero_grad()
            batch_loss.backward()
            optimizer.step()

            # print(
            #     "Train loss: {:.3f}, Train avg error: {:.3f}"
            #         .format(criterion(output, target), accuracy_sigmoid(output, target)))


        net.eval()
        val_loss, val_accs, val_err, val_length = [0, 0, 0, 0]

        for val_batch in val_iter:
            users, (docs, lengths), ratings = val_batch.user, val_batch.doc_title, val_batch.ratings
            batch_without_negative_sampling = {'user': users, 'doc_title': docs}
            val_output = net(SimpleNamespace(**batch_without_negative_sampling),).reshape(-1)
            val_target = ratings.float().to(device)
            val_loss += criterion(val_output, val_target) * val_batch.batch_size
            val_err += accuracy_sigmoid(val_output, val_target) * val_batch.batch_size
            val_accs += accuracy_sigmoid(val_output, val_target) * val_batch.batch_size
            val_length += val_batch.batch_size

        val_loss /= val_length
        val_err /= val_length
        val_accs /= val_length
        val_res.append(val_accs)

        print(
            "Epoch {}: Train loss: {:.2f},  Train accs: {:.2f}, Train avg error: {:.2f}"
                .format(i+1, np.mean(train_loss), 1.0 - np.mean(train_accs), np.mean(train_error)))
        print(
            "          Validation loss: {:.2f}, Validation accs: {:.2f}, Validation avg error: {:.2f}"
                .format(val_loss, 1.0 - val_accs, val_err))
        print()


def accuracy_sigmoid(output, target):
    return torch.mean(torch.abs(output - target).float()).cpu().data.numpy()


def accuracy(output, target):
    return torch.mean(torch.abs(torch.round(output) - target)).cpu().data.numpy()


def get_numpy(loss):
    return loss.cpu().data.numpy()


In [0]:
# MAIN

from torch import optim, nn

citeulike = citeulike(batch_size=20)

train_iter = citeulike.train_iter
test_iter = citeulike.test_iter
validation_iter = citeulike.validation_iter

user_field = citeulike.user
title_field = citeulike.doc_title

print('Data loaded.')

Device: cuda:0


.vector_cache/wiki.simple.vec: 293MB [00:04, 62.9MB/s]                           
100%|█████████▉| 110663/111051 [00:14<00:00, 7823.40it/s]

Data loaded.


In [0]:
#Simple Collaborative filtering model

net = CollaborativeFilteringNet(article_field=title_field, user_field=user_field, author_dim=50).to(device)
#opt = optim.Adam(net.parameters(), lr=1e-4, weight_decay=1e-6)
opt = optim.SGD(net.parameters(), lr=1e-2, momentum=0.8)
criterion = nn.BCELoss()
train_with_negative_sampling(train_iter=train_iter, test_iter=test_iter, val_iter=validation_iter,
                                net=net, optimizer=opt, criterion=criterion, num_epochs=30)

Epoch 1: Train loss: 0.69,  Train accs: 0.57, Train avg error: 0.50
          Validation loss: 0.67, Validation accs: 0.52, Validation avg error: 0.48

Epoch 2: Train loss: 0.65,  Train accs: 0.62, Train avg error: 0.47
          Validation loss: 0.63, Validation accs: 0.57, Validation avg error: 0.43

Epoch 3: Train loss: 0.64,  Train accs: 0.64, Train avg error: 0.45
          Validation loss: 0.62, Validation accs: 0.58, Validation avg error: 0.42

Epoch 4: Train loss: 0.63,  Train accs: 0.65, Train avg error: 0.44
          Validation loss: 0.61, Validation accs: 0.58, Validation avg error: 0.42

Epoch 5: Train loss: 0.63,  Train accs: 0.65, Train avg error: 0.44
          Validation loss: 0.60, Validation accs: 0.58, Validation avg error: 0.42

Epoch 6: Train loss: 0.62,  Train accs: 0.66, Train avg error: 0.44
          Validation loss: 0.62, Validation accs: 0.58, Validation avg error: 0.42

Epoch 7: Train loss: 0.62,  Train accs: 0.66, Train avg error: 0.43
          Validation

In [21]:
# Collaborative filtering with LSTM

net = LstmNet(article_field=title_field, user_field=user_field).to(device)
opt = optim.Adam(net.parameters(), lr=1e-3, weight_decay=1e-6)
#opt = optim.SGD(net.parameters(), lr=1e-1, momentum=0.8)
criterion = nn.BCELoss()
train_with_negative_sampling(train_iter=train_iter, test_iter=test_iter, val_iter=validation_iter,
                                net=net, optimizer=opt, criterion=criterion, num_epochs=30)

Epoch 1: Train loss: 0.59,  Train accs: 0.68, Train avg error: 0.41
          Validation loss: 0.42, Validation accs: 0.70, Validation avg error: 0.30

Epoch 2: Train loss: 0.54,  Train accs: 0.72, Train avg error: 0.36
          Validation loss: 0.38, Validation accs: 0.73, Validation avg error: 0.27

Epoch 3: Train loss: 0.50,  Train accs: 0.75, Train avg error: 0.33
          Validation loss: 0.37, Validation accs: 0.74, Validation avg error: 0.26

Epoch 4: Train loss: 0.47,  Train accs: 0.77, Train avg error: 0.31
          Validation loss: 0.33, Validation accs: 0.77, Validation avg error: 0.23

Epoch 5: Train loss: 0.45,  Train accs: 0.78, Train avg error: 0.29
          Validation loss: 0.32, Validation accs: 0.79, Validation avg error: 0.21

Epoch 6: Train loss: 0.43,  Train accs: 0.80, Train avg error: 0.28
          Validation loss: 0.30, Validation accs: 0.81, Validation avg error: 0.19

Epoch 7: Train loss: 0.41,  Train accs: 0.81, Train avg error: 0.26
          Validation

In [0]:
len(title_field.vocab.freqs)

13483